# KMeans Benchmark: Scitkit, Pilot and Spark/MLlib


This is perhaps the best known database to be found in the pattern recognition literature. The data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant (see <https://archive.ics.uci.edu/ml/datasets/Iris>). 

Source: R. A. Fisher, The Use of Multiple Measurements in Taxonomic Problems, 1936, http://rcs.chemometrics.ru/Tutorials/classification/Fisher.pdf

Pictures (Source [Wikipedia](https://en.wikipedia.org/wiki/Iris_flower_data_set))

<table>
<tr><td>
Setosa
</td><td>
Versicolor
</td><td>
Virginica
</td></tr>
<tr><td>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg/180px-Kosaciec_szczecinkowaty_Iris_setosa.jpg"/> 
</td><td>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Iris_versicolor_3.jpg/320px-Iris_versicolor_3.jpg"/>
</td><td>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Iris_virginica.jpg/295px-Iris_virginica.jpg"/>
</td></tr></table>

## Data Overview

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/pydata/pandas/master/pandas/tests/data/iris.csv")
data.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## Scikit

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
results = kmeans.fit_predict(data[['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']])

In [ ]:
data_kmeans=pd.concat([data, pd.Series(results, name="ClusterId")], axis=1)
data_kmeans.head()

## Spark MLLib KMeans

In [ ]:
from numpy import array
from math import sqrt

%run ../env.py
%run ../util/init_spark.py

from pilot_hadoop import PilotComputeService as PilotSparkComputeService

pilotcompute_description = {
    "service_url": "yarn-client://sc15.radical-cybertools.org",
    "number_of_processes": 5
}
pilot_spark = PilotSparkComputeService.create_pilot(pilotcompute_description=pilotcompute_description)
sc = pilot_spark.get_spark_context()

In [ ]:
from pyspark.mllib.clustering import KMeans, KMeansModel
# Load and parse the data

# index for points
idx = []
for i in range(1000000):
    idx.extend([i]*3)    
idx_rdd = sc.parallelize(idx, numSlices=100)

# 3D points stored on 3 lines
data = sc.textFile("/data/kmeans/dataset_1M_3d.in")
parsedData = data.map(lambda x: float(x))

In [ ]:
parsedData.count()

In [ ]:
# merge data
parsedData = parsedData.repartition(1)
idx_rdd = idx_rdd.repartition(1)
rdd = idx_rdd.zip(parsedData)

In [ ]:
input_data = rdd.groupByKey().mapValues(list).values()
input_data = input_data.repartition(100)

In [ ]:
input_data.count()

In [ ]:
input_data.getNumPartitions()

In [ ]:
start = time.time()
# Build the model (cluster the data)
clusters = KMeans.train(input_data, 50, maxIterations=10, runs=1, initializationMode="random")
end = time.time()
print "Training Time %.2f"%(end-start)

In [ ]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = input_data.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))


In [ ]:
str(clusters)

## Pilot-based Approach

In [ ]:
import pandas as pd
import sklearn
from sklearn.metrics.pairwise import euclidean_distances

def mapper(data, centroids):
    # compute distances between all points and centroids
    distance = sklearn.metrics.pairwise.euclidean_distances(data, clusters)
    # compute cluster with min distance
    cluster_id = np.argmin(distance, axis=1)
    # reshape to row vector
    cluster_id = cluster_id[:, np.newaxis]
    # join data and cluster ids
    data=np.column_stack((data, cluster_id))
    return data


def compute_new_centroids(distances):
    df = pd.DataFrame(distances)
    df[3] =  df[3].astype(int)
    df = df.groupby(3)[0,1,2].mean()
    centroids_np = df.as_matrix()
    return centroids_np

In [ ]:
start = time.time()
clusters = data[np.random.choice(data.shape[0], 50, replace=False),:]
for i in range(10):
    points_cluster = mapper(data, clusters) 
    new_centroids = compute_new_centroids(points_cluster)
    clusters=new_centroids
end = time.time()
print "KMeans Compute Time: %.1f"%(end-start)